# Assignment 1

In this assignment, you'll be working with messy medical data and using regex to extract relevant infromation from the data. 

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal of this assignment is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

Once you have extracted these date patterns from the text, the next step is to sort them in ascending chronological order accoring to the following rules:
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).
* Watch out for potential typos as this is a raw, real-life derived dataset.

With these rules in mind, find the correct date in each note and return a pandas Series in chronological order of the original Series' indices.

For example if the original series was this:

    0    1999
    1    2010
    2    1978
    3    2015
    4    1985

Your function should return this:

    0    2
    1    4
    2    0
    3    1
    4    3

Your score will be calculated using [Kendall's tau](https://en.wikipedia.org/wiki/Kendall_rank_correlation_coefficient), a correlation measure for ordinal data.

*This function should return a Series of length 500 and dtype int.*

In [4]:
import pandas as pd
import datetime

#defining function for returning months in integers
def monthTonum(shortMonth):
    return{
        'jan' : 1,'feb' : 2,'mar' : 3,'apr' : 4,'may' : 5,'jun' : 6,
        'jul' : 7,'aug' : 8,'sep' : 9,'oct' : 10,'nov' : 11,'dec' : 12
    }[shortMonth]

doc = []
with open('dates.txt') as file:
    for line in file:
        doc.append(line.lower())

df = pd.Series(doc)
df.head()

0         03/25/93 total time of visit (in minutes):\n
1                       6/18/85 primary care doctor:\n
2    sshe plans to move as of 7/8/71 in-home servic...
3                7 on 9/27/75 audit c score current:\n
4    2/6/96 sleep studypain treatment pain level (n...
dtype: object

In [5]:
def date_sorter():
 
    # finding first possible format - all digits.
    df['dates'] = df.str.extract(r'(\d?\d)[./-]+(\d?\d)[./-]+(\d?\d?\d{2})')
    df['dates'].dropna(inplace = True)
    df['dates'] = df['dates'].astype(int)
 
    # --> To check columns with values more than 31.
    #df['dates']['check1'] = df['dates'][df['dates']>31].count(axis=1)
    #df['dates'][df['dates']['check1']>1]
    df['dates']['yy'] = df['dates'].values.max(axis=1)
 
    # --> Assume [0] is month, [1] is day.
    df['dates']['cleaned_d'] = df['dates'][1]
    df['dates']['cleaned_m'] = df['dates'][0]
    for i, row in df['dates'].iterrows():
        if row['yy'] < 100:
            df['dates'].loc[i,'cleaned_yyyy'] = 1900 + row['yy']
        else:
            df['dates'].loc[i,'cleaned_yyyy'] = row['yy']
   
    # finding day-mmm-year dates
    df['dates1'] = df.str.extract(r'(\d?\d)[st|nd|rd|th]?[- ,.]*?(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[- ,.]*?(\d?\d?\d{2})')
    df['dates1'] = df['dates1'].dropna()
 
    df['dates1'][[0,2]] = df['dates1'][[0,2]].astype(int)
    # df['dates1'][2]<1900 yielded no results; all were in yyyy format.
    # df['dates1'][0]>31 yielded no results.
    df['dates1']['cleaned_d'] = df['dates1'][0]
    for i, rows in df['dates1'].iterrows():
        df['dates1'].loc[i, 'cleaned_m'] = monthTonum(rows[1])
        df['dates1']['cleaned_yyyy'] = df['dates1'][2]
 
    #finding mmm-day-year dates
    df['dates2'] = df.str.extract(r'(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[- ,.][- ,.]?(\d?\d)[st|nd|rd|th]?[- ,.]?[- ,$.]?(\d{4})')
    df['dates2'] = df['dates2'].dropna()
    df['dates2'][[1,2]] = df['dates2'][[1,2]].astype(int)
    df['dates2']['cleaned_d'] = df['dates2'][1]
    for i, rows in df['dates2'].iterrows():
        df['dates2'].loc[i, 'cleaned_m'] = monthTonum(rows[0])
    df['dates2']['cleaned_yyyy'] = df['dates2'][2]
 
    dfgt = df['dates1'][['cleaned_d','cleaned_m','cleaned_yyyy']].append(df['dates2'][['cleaned_d','cleaned_m','cleaned_yyyy']]).append(df['dates'][['cleaned_d','cleaned_m','cleaned_yyyy']])
   
    #finding mmm-year dates; assuming day==1 in remaining lines
    df['dates3'] = df[([i for i in range(500) if i not in dfgt.index.tolist()])].str.extract(r'[\D{3}]?(jan|feb|mar|apr|may|jun|jul|aug|sep|oct|nov|dec)[a-z]*[- ,.]*?(\d{4})')
    df['dates3'] = df['dates3'].dropna()
    for i, row in df['dates3'].iterrows():
        df['dates3'].loc[i,'cleaned_d'] = 1
        df['dates3'].loc[i,'cleaned_m'] = monthTonum(row[0])
        df['dates3'].loc[i,'cleaned_yyyy'] = row[1]
    dfgt = dfgt.append(df['dates3'][['cleaned_d','cleaned_m','cleaned_yyyy']])
   
    #finding mm/yyyy; assuming day==1
    df4 = df[[i for i in range(500) if i not in dfgt.index.tolist()]].str.extract(r'(\d?\d)/(\d{4})')
    df4 = df4.dropna()
    df4['cleaned_d'] = 1
    df4['cleaned_m'] = df4[0].astype(int)
    df4['cleaned_yyyy'] = df4[1].astype(int)
 
    dfgt = dfgt.append(df4[['cleaned_d','cleaned_m','cleaned_yyyy']])
   
    #finding yyyy only; assuming day==1, month==1
    df5 = pd.DataFrame()
    df5['cleaned_yyyy'] = df[[i for i in range(500) if i not in dfgt.index.tolist()]].str.extract(r'(\d{4})')
    df5['cleaned_d'] = 1
    df5['cleaned_m'] = 1
 
    dfgt = dfgt.append(df5[['cleaned_d','cleaned_m','cleaned_yyyy']])
 
    dfgt = dfgt.astype(int)
    for i, rows in dfgt.iterrows():
        dfgt.loc[i,'cleaned_date'] = datetime.datetime(year=rows['cleaned_yyyy'], month=rows['cleaned_m'], day=rows['cleaned_d'])
   
    return dfgt.sort_values(by='cleaned_date').index
 
date_sorter()

Int64Index([  9,  84,   2,  53,  28, 474, 153,  13, 129,  98,
            ...
            152, 235, 464, 253, 231, 427, 141, 186, 161, 413],
           dtype='int64', length=500)